In [ ]:
import pandas as pd
from glob import glob
import re
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import xmltodict

In [ ]:
r = requests.get("https://www.resultats-elections.interieur.gouv.fr/telechargements/LG2022/resultatsT1/").content
soup = BeautifulSoup(r)
soup = BeautifulSoup(r)
dep_path_list = [i["href"] for i in soup.find_all("a")[1:-1] ]
dep_path_list

In [ ]:


def get_data(com):
    b = com["Tours"]["Tour"]["Resultats"]["Candidats"]["Candidat"]
    result = pd.json_normalize(b)


    a = com["Tours"]["Tour"]["Mentions"]
    meta = pd.json_normalize(a)[["Inscrits.Nombre","Abstentions.Nombre","Votants.Nombre","Blancs.Nombre","Nuls.Nombre"]]
    meta.columns = [i.replace(".Nombre","").lower() for i in meta.columns]
    for col in meta.columns:
        result[col] = meta[col][0]
    result["code_commune"] =  com["CodSubCom"]
    result["commune"] =  com["LibSubCom"]
    result["code_circo"] =  com["CodCirLg"]
    result = result.rename(columns={"NbVoix" : "voix","NomPsn" : "nom", "PrenomPsn" : "prenom", "CivilitePsn" : "civilite", "CodNua" : "code_nuance", "LibNua" : "nuance"})
    keep_cols = [
                 "code_circo",
                 "code_commune",
                 "commune",
                 'inscrits',
                 "votants",
                 "blancs",
                 "nuls",
                 "nom",
                 "prenom",
                 "civilite",
                 "code_nuance",
                 "nuance",
                 "voix"
                ]
    result = result[keep_cols]
    return result


In [ ]:
dep_df_list = []
base_url = "https://www.resultats-elections.interieur.gouv.fr/telechargements/LG2022/resultatsT1/"
for dep in dep_path_list:
    print(dep)
    dep_url = base_url+ dep
    com_path = dep.replace("/","") + "com.xml"
    url = dep_url + com_path

    r = requests.get(url).content
    j = xmltodict.parse(r)
    j = json.dumps(j)
    j = json.loads(j)

    com_list = j["Election"]["Departement"]["Communes"]["Commune"]

    df_list = []
    try:
        for com in com_list:
            df = get_data(com)
            df_list.append(df)
        df = pd.concat(df_list)
    except:
        df = get_data(com_list)
    df.insert(0,"dep",dep.replace("/",""))
    dep_df_list.append(df)

df = pd.concat(dep_df_list)

def shorten_dep(dep):
    if dep[0] == "0":
        dep = dep[1:]
    return dep

df["dep"] = df["dep"].apply(shorten_dep)
df.to_csv("results.csv",index=False)

for col in ["inscrits","votants","blancs","nuls","voix"]:
    df[col] = df[col].astype("int32")
